In [ ]:
# Mariana's script to build cluster
from ase.build import fcc111, add_adsorbate
from ase.io import read, write
from ase import Atoms
import numpy
import sys

def get_dist(point1, point2):
    a=numpy.array(point1)
    b=numpy.array(point2)
    return numpy.sqrt(numpy.sum((a-b)**2))

def _main(dcut,shrink):
    lattice=3.6318 # number for Cu light
    interlayer=lattice/numpy.sqrt(3) # interlayer spacing for fcc(111)
    height=2.0 # height of adsorbate -- it is in fact not really important and oculd have done without
    slab=fcc111('Cu', a=3.6318, size=(8,8,8), vacuum=50.0)
    add_adsorbate(slab, 'H', height, 'ontop', offset=4) # this is here just to get position automatically. Can be substituted by the actual COM pos. of the relaxed structure
    center=slab.get_positions()[-1] # here I get the position of the adsorbate
    poscu=numpy.array([i.position for i in slab if i.symbol=="Cu"]) # here I get all positions of Cu
    newpos=[]
    for i in poscu:
        xy=get_dist(i[:2], center[:2]) #this is just xy distance from a center
       #z=abs(i[2]-center[2]) # change to call for get_dist
        z=get_dist(i[2], center[2])
        if (xy < dcut and (z <= (0.1*interlayer+height))): # first layer
            newpos.append(i)
        if (xy < (1.0-shrink)*dcut and (z > (0.1*interlayer+height)) and (z <= (1.1*interlayer+height))): # second layer
            newpos.append(i)
        if (xy < (1.0-2*shrink)*dcut and (z > (1.1*interlayer+height)) and (z <= (2.1*interlayer+height))): # third layer
            newpos.append(i)
        if (xy < (1.0-3*shrink)*dcut and (z > (2.1*interlayer+height)) and (z <= (3.1*interlayer+height))): # fourth layer
            newpos.append(i)
        if (xy < (1.0-4*shrink)*dcut and (z > (3.1*interlayer+height)) and (z <= (4.1*interlayer+height))): # fourth layer
            newpos.append(i)
        # The ifs above can be generalized for a loop or at least increased for more layers
    newpos=numpy.array(newpos)
    n=int(len(newpos))
    cluster=Atoms(positions=newpos)
    cluster.set_chemical_symbols("Cu"+str(n))
    write('new-geo.in', cluster, format="aims")
    write('originalslab.in', slab, format="aims")


if __name__ == "__main__":
      _main(float(sys.argv[1]), float(sys.argv[2]))
# first argument, read as dcut, is the radial distance at the first layer
# second argument, read as shrink, is how much you shrink this distance from layer to layer 


In [ ]:
# my cluster building script 
from ase.build import fcc111, add_adsorbate
from ase.io import read, write
from ase import Atoms
import numpy
import sys
from ase.build import make_supercell
from pylab import savetxt, transpose

def get_dist(point1, point2):
    a=numpy.array(point1)
    b=numpy.array(point2)
    return numpy.linalg.norm(a-b)

def _main(dcut,shrink):

    slab=read("geometry.in.next_step", format='aims')
    lattice=(slab.get_cell_lengths_and_angles()[1])/2
    center=[33.34239,19.2466, 63.0828833] #center of the first layer 
    mol=numpy.array([i.position for i in slab if i.symbol=="C" or i.symbol=="H"])
    benzene=Atoms('C6H6',positions=mol)
    COM=benzene.get_center_of_mass(scaled=False)
    print(COM)
    slab = make_supercell(slab, numpy.diag([6,6,1]))
    poscu=numpy.array([i.position for i in slab if i.symbol=="Cu"])
   # add_adsorbate(slab, 'H', 3.0127533, position=[33.34239,19.2466] ) # not working
   # write('whatslab.in', slab, format="aims")



    interlayer=lattice/numpy.sqrt(3) # interlayer spacing for fcc(111)
    height=3.0127533
    newpos=[]
      xy=get_dist(i[:2], center[:2]) #this is just xy distance from a center
       #z=abs(i[2]-center[2]) # change to call for get_dist
        z=get_dist(i[2], center[2])
        if (xy <(1.0-0.05*shrink)*dcut and (z <= (0.1*interlayer+height))): # first layer
            newpos.append(i)
        if (xy < (0.9-0.1*shrink)*dcut and (z > (0.1*interlayer+height)) and (z <= (1.1*interlayer+height))): # second layer
            newpos.append(i)
        if (xy < (0.3-0.2*shrink)*dcut and (z > (1.1*interlayer+height)) and (z <= (2.1*interlayer+height))): # third layer
            newpos.append(i)
        if (xy < (0.007-0.3*shrink)*dcut and (z > (2.1*interlayer+height)) and (z <= (3.1*interlayer+height))): # fourth layer
            newpos.append(i)
         #The ifs above can be generalized for a loop or at least increased for more layers
        #if (xy < (1.0-4*shrink)*dcut and (z > (3.1*interlayer+height)) and (z <= (4.1*interlayer+height))): # fifth layer
         #   newpos.append(i)
    newpos=numpy.array(newpos)
    n=int(len(newpos))
    cluster=Atoms(positions=newpos)
    cluster.set_chemical_symbols("Cu"+str(n))
    vx=COM[:2]
    v=numpy.append(vx,height)
    V=Atoms(positions=[v])
    benzene.translate([33.34239,19.2466, height]-V.positions)
    cluster.extend(benzene)
    write('new-geo.in', cluster, format="aims")
    write('originalslab.in', slab, format="aims")


if __name__ == "__main__":
      _main(float(sys.argv[1]), float(sys.argv[2]))
# first argument, read as dcut, is the radial distance at the first layer
# second argument, read as shrink, is how much you shrink this distance from layer to layer

